# Streamlit Operations Reference Guide

This notebook serves as a comprehensive reference for Streamlit operations that you can use in your projects. It demonstrates all key features with well-documented examples.

## Table of Contents
1. [Setup and Installation](#setup)
2. [Page Configuration](#page-config)
3. [Custom Styling](#styling)
4. [Authentication System](#auth)
5. [Sidebar Components](#sidebar)
6. [Data Operations](#data-ops)
7. [Visualization](#viz)
8. [Layout Components (Tabs, Expanders)](#layout)
9. [Date and Time Components](#datetime)
10. [Input Widgets](#input-widgets)
11. [Form Handling](#forms)
12. [File Operations (Upload, Download)](#file-ops)
13. [Session State Management](#session-state)
14. [Caching](#caching)
15. [Progress Indicators](#progress)
16. [Component Embedding (HTML, iframes)](#embed)
17. [Theme Customization](#theme)
18. [Advanced Admin UI](#admin)

<a id='setup'></a>
## 1. Setup and Installation

First, install the required dependencies.

In [1]:
# Install required packages
# pip install streamlit pandas numpy matplotlib plotly pillow

## Overview of the Reference Application

The code below creates a comprehensive Streamlit application that demonstrates all the key features. Each section is well-commented to explain the functionality.

You can run the complete application Or copy specific sections for your own projects.

In [2]:
%%writefile streamlit_application.py

# ===================================
# STREAMLIT REFERENCE APPLICATION
# ===================================
# A comprehensive demonstration of Streamlit features
# Organized for easy reference and reuse

# -----------------------------
# IMPORT REQUIRED LIBRARIES
# -----------------------------
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import json
import time
import datetime
from datetime import datetime, date, timedelta
import os
import base64
import io
import random
from PIL import Image
import hashlib
import sqlite3
from functools import wraps


# -----------------------------
# USER AUTHENTICATION SYSTEM
# -----------------------------
# Uses JSON file to store user credentials

# File path for user data
USERS_FILE = 'streamlit_users.json'

# User management functions
def load_users():
    """Load users from JSON file, create default users if file doesn't exist"""
    if os.path.exists(USERS_FILE):
        try:
            with open(USERS_FILE, 'r') as file:
                users = json.load(file)
            return users
        except (json.JSONDecodeError, IOError):
            # File exists but is corrupted, create new with defaults
            return create_default_users()
    else:
        # File doesn't exist, create default users
        return create_default_users()

def create_default_users():
    """Create default users and save to file"""
    default_users = {
        "admin": {
            "password": "admin",
            "role": "admin",
            "email": "admin@example.com"
        },
        "user": {
            "password": "user",
            "role": "user",
            "email": "user@example.com"
        }
    }
    save_users(default_users)
    return default_users

def save_users(users):
    """Save users dictionary to JSON file"""
    with open(USERS_FILE, 'w') as file:
        json.dump(users, file, indent=4)

def add_user_to_file(username, password, role, email=""):
    """Add a new user to the JSON file"""
    users = load_users()
    
    # Check if username already exists
    if username in users:
        return False
    
    # Add new user
    users[username] = {
        "password": password,
        "role": role,
        "email": email
    }
    
    # Save updated users
    save_users(users)
    return True

def verify_file_user(username, password):
    """Verify user credentials against the JSON file"""
    users = load_users()
    
    if username not in users:
        return False, ""
    
    user_data = users[username]
    if user_data["password"] == password:
        return True, user_data["role"]
    
    return False, ""

# Email sending simulation for admin panel
def send_dummy_email(recipients, subject, message):
    """Simulate sending an email (just returns a success message)"""
    # In a real application, you would connect to an SMTP server
    # For now, we'll just simulate it
    
    # Create a timestamp for the "sent" email
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Store the "sent" email in session state for display
    if 'sent_emails' not in st.session_state:
        st.session_state.sent_emails = []
    
    st.session_state.sent_emails.append({
        "timestamp": timestamp,
        "recipients": recipients,
        "subject": subject,
        "message": message
    })
    
    # Return success message
    return f"Email sent to {len(recipients)} recipient(s) at {timestamp}"
    
# Initialize by loading users when the app starts
USERS = load_users()

# Authentication functions
def verify_password(username, password):
    """Verify password against stored value in JSON file."""
    authenticated, _ = verify_file_user(username, password)
    return authenticated

def authenticate():
    """Authentication flow for the application."""
    # Initialize session state variables for authentication
    if 'authenticated' not in st.session_state:
        st.session_state.authenticated = False
    if 'username' not in st.session_state:
        st.session_state.username = ""
    if 'role' not in st.session_state:
        st.session_state.role = ""
    
    if not st.session_state.authenticated:
        # Display login form in a container to make it look nicer
        login_container = st.container()
        with login_container:
            st.markdown("<h2 class='section-header'>Login</h2>", unsafe_allow_html=True)
            
            # Using columns to center the login form
            col1, col2, col3 = st.columns([1, 2, 1])
            with col2:
                username = st.text_input("Username")
                password = st.text_input("Password", type="password")
                
                if st.button("Login"):
                    authenticated, role = verify_file_user(username, password)
                    if authenticated:
                        st.session_state.authenticated = True
                        st.session_state.username = username
                        st.session_state.role = role
                        st.rerun()
                    else:
                        st.error("Invalid username or password")
                        
                st.markdown("Try with username: 'admin' and password: 'admin' or username: 'user' and password: 'user'")
        
        return False  # Not authenticated
    
    return True  # Already authenticated

# Role-based access control decorators
def require_auth(func):
    """Decorator to require authentication for a function."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        if st.session_state.get('authenticated', False):
            return func(*args, **kwargs)
        else:
            st.warning("You need to log in to access this feature")
            return None
    return wrapper

def require_admin(func):
    """Decorator to require admin role for a function."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        if st.session_state.get('authenticated', False) and st.session_state.get('role', '') == 'admin':
            return func(*args, **kwargs)
        else:
            st.warning("You need admin privileges to access this feature")
            return None
    return wrapper


# -----------------------------
# PAGE CONFIGURATION
# -----------------------------
st.set_page_config(
    page_title="Streamlit Reference App",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'Get Help': 'https://docs.streamlit.io',
        'Report a bug': "mailto:support@yourcompany.com",
        'About': "# This app contains reference implementations of all Streamlit features."
    }
)


# -----------------------------
# CUSTOM CSS STYLING
# -----------------------------
custom_css = """
<style>
    .main-header {
        font-size: 2.5rem;
        color: #4CAF50;
        text-align: center;
    }
    
    .section-header {
        font-size: 1.8rem;
        color: #2196F3;
        padding-top: 1rem;
    }
    
    .sub-section {
        background-color: #f5f5f5;
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 1rem 0;
    }
    
    .highlight {
        background-color: #ffd54f;
        padding: 0.2rem;
        border-radius: 0.2rem;
    }
    
    .sidebar-content {
        padding: 1rem;
    }
    
    /* Custom button styling */
    .stButton button {
        background-color: #4CAF50;
        color: white;
        border-radius: 0.5rem;
    }
    
    /* Custom metric styling */
    .stMetric {
        background-color: #e3f2fd;
        padding: 1rem;
        border-radius: 0.5rem;
    }
</style>
"""

st.markdown(custom_css, unsafe_allow_html=True)


# -----------------------------
# DATA GENERATION AND PROCESSING
# -----------------------------
def generate_sample_data():
    """Generate a sample DataFrame for demonstration purposes."""
    # For demo purposes, create sample data
    # In a real application, replace this with your actual data source
    try:
        # Try to read from a CSV file if it exists
        df = pd.read_csv("data.csv")
    except FileNotFoundError:
        # Generate random data if file doesn't exist
        df = pd.DataFrame({
            'Date': pd.date_range(start='2025-01-01', periods=100, freq='D'),
            'Category': np.random.choice(['A', 'B', 'C', 'D'], 100),
            'Region': np.random.choice(['North', 'South', 'East', 'West'], 100),
            'Sales': np.random.randint(100, 1000, 100),
            'Profit': np.random.randint(-100, 500, 100),
            'Customers': np.random.randint(10, 100, 100),
            'Latitude': np.random.uniform(20, 50, 100),
            'Longitude': np.random.uniform(-130, -70, 100)
        })
    return df

# Sample function to demonstrate caching
@st.cache_data  # Caching decorator for better performance
def process_data(df, operation):
    """Simulates processing data with caching for performance."""
    # Simulate processing time
    time.sleep(1)
    
    if operation == 'sum':
        return df.sum(numeric_only=True)
    elif operation == 'mean':
        return df.mean(numeric_only=True)
    elif operation == 'count':
        return df.count()
    return df.describe()

# File download utility
def download_csv(df, filename="data.csv"):
    """Creates a download link for a DataFrame."""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">Download {filename}</a>'
    return href


# -----------------------------
# MAIN APPLICATION
# -----------------------------
def main():
    # Authentication check
    if not authenticate():
        return  # Stop here if not authenticated
    
    # App header
    st.markdown("<h1 class='main-header'>Streamlit Reference Application</h1>", unsafe_allow_html=True)
    
    # User info and logout button
    col1, col2 = st.columns([3, 1])
    with col1:
        st.markdown(f"Welcome, **{st.session_state.username}** (Role: {st.session_state.role})")
    with col2:
        if st.button("Logout"):
            st.session_state.authenticated = False
            st.session_state.username = ""
            st.session_state.role = ""
            st.rerun()
    
    # Load sample data
    sample_data = generate_sample_data()
    
    # Initialize widget states for responsive UI
    def ensure_widget_state():
        """Initialize all widget states to ensure first-click responsiveness"""
        # List all widget keys that need initialization
        widget_keys = [
            "sidebar_categories_key", "screener_categories_key", 
            "viz_type_radio", "sales_range_slider", "calc_column",
            "calc_operation"
        ]
        
        # Initialize any missing widget states
        for key in widget_keys:
            if key not in st.session_state:
                if "categories" in key:
                    st.session_state[key] = list(sample_data['Category'].unique())
                elif key == "viz_type_radio":
                    st.session_state[key] = "Bar Chart"
                elif key == "calc_operation":
                    st.session_state[key] = "Sum"
                elif key == "calc_column":
                    st.session_state[key] = sample_data.select_dtypes(include=['number']).columns[0]
    
    # Initialize widget states
    ensure_widget_state()
    
    # -----------------------------
    # SIDEBAR IMPLEMENTATION
    # -----------------------------
    with st.sidebar:
        st.markdown("<div class='sidebar-content'>", unsafe_allow_html=True)
        st.markdown("<h2>Control Panel</h2>", unsafe_allow_html=True)
        
        # Theme selection with full page styling
        st.subheader("Theme Settings")
        
        # Initialize theme color in session state if not present
        if 'theme_color' not in st.session_state:
            st.session_state.theme_color = "#4CAF50"  # Default green
        
        # Color picker for theme selection
        theme_color = st.color_picker("Select Theme Color", st.session_state.theme_color)
        st.session_state.theme_color = theme_color
        
        # Function to get darker and lighter versions of the theme color
        def adjust_color_brightness(hex_color, factor):
            """Adjust color brightness by factor (0-2, where <1 is darker, >1 is lighter)"""
            # Convert hex to RGB
            hex_color = hex_color.lstrip('#')
            r, g, b = int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)
            
            # Adjust brightness
            r = min(255, max(0, int(r * factor)))
            g = min(255, max(0, int(g * factor)))
            b = min(255, max(0, int(b * factor)))
            
            # Convert back to hex
            return f"#{r:02x}{g:02x}{b:02x}"
        
        # Generate color variations
        darker_color = adjust_color_brightness(theme_color, 0.8)  # 20% darker
        darkest_color = adjust_color_brightness(theme_color, 0.6)  # 40% darker
        lighter_color = adjust_color_brightness(theme_color, 1.2)  # 20% lighter
        lightest_color = adjust_color_brightness(theme_color, 1.4)  # 40% lighter
        
        # Create a theme preview
        st.markdown(f"""
            <div style="
                margin-top: 10px;
                padding: 10px;
                border-radius: 5px;
                background-color: {theme_color};
                color: white;
                text-align: center;
                font-weight: bold;
            ">
                Theme Preview
            </div>
        """, unsafe_allow_html=True)
        
        # Apply comprehensive theme styling
        st.markdown(f"""
            <style>
            /* Main elements */
            .stApp {{
                background-color: white;
            }}
            
            /* Sidebar */
            section[data-testid="stSidebar"] {{
                background-color: #f8f9fa;
                border-right: 2px solid {lighter_color};
            }}
            
            /* Headers */
            h1, h2, h3, h4, h5, h6 {{
                color: {darker_color} !important;
            }}
            
            /* Buttons */
            .stButton button {{
                background-color: {theme_color} !important;
                color: white !important;
                border: none !important;
                border-radius: 4px !important;
                padding: 4px 12px !important;
            }}
            .stButton button:hover {{
                background-color: {darker_color} !important;
            }}
            
            /* Progress bar */
            .stProgress > div > div > div > div {{
                background-color: {theme_color} !important;
            }}
            
            /* Slider */
            .stSlider div[data-baseweb="slider"] div[data-testid="stThumbValue"] {{
                background-color: {theme_color} !important;
            }}
            
            /* Checkbox and Radio */
            .stCheckbox label span[data-testid="stMarkdownContainer"] p {{
                color: {darker_color} !important;
            }}
            .stRadio label span[data-testid="stMarkdownContainer"] p {{
                color: {darker_color} !important;
            }}
            
            /* Selectbox */
            div[data-baseweb="select"] {{
                border-color: {theme_color} !important;
            }}
            
            /* Multiselect */
            .stMultiSelect span[data-baseweb="tag"] {{
                background-color: {lighter_color} !important;
            }}
            
            /* Tabs */
            .stTabs [data-baseweb="tab-list"] button[aria-selected="true"] {{
                border-bottom-color: {theme_color} !important;
                color: {darker_color} !important;
            }}
            .stTabs [data-baseweb="tab-list"] button:hover {{
                color: {theme_color} !important;
            }}
            
            /* Expanders */
            .streamlit-expanderHeader {{
                color: {darker_color} !important;
                font-weight: bold;
            }}
            
            /* Metric */
            .stMetric label {{
                color: {darker_color} !important;
            }}
            
            /* Text inputs and text areas */
            .stTextInput input, .stTextArea textarea {{
                border-color: {lighter_color} !important;
            }}
            .stTextInput input:focus, .stTextArea textarea:focus {{
                border-color: {theme_color} !important;
                box-shadow: 0 0 0 3px {lightest_color} !important;
            }}
            
            /* Dataframe */
            .stDataFrame div[data-testid="stTable"] {{
                border: 1px solid {lighter_color} !important;
            }}
            .stDataFrame th {{
                background-color: {lighter_color} !important;
                color: {darkest_color} !important;
            }}
            
            /* Main headers defined in custom CSS */
            .main-header {{
                color: {theme_color} !important;
            }}
            
            .section-header {{
                color: {darker_color} !important;
            }}
            
            /* Custom elements from your CSS */
            .sub-section {{
                background-color: {lightest_color} !important;
                border-left: 4px solid {theme_color} !important;
            }}
            
            .highlight {{
                background-color: {lighter_color} !important;
            }}
            </style>
        """, unsafe_allow_html=True)
        
        # Data operations section
        st.subheader("Data Operations")
        
        # Category filtering
        category_options = list(sample_data['Category'].unique())
        
        # Initialize session state for categories
        if 'current_categories' not in st.session_state:
            st.session_state.current_categories = category_options.copy()
        
        # Update categories when selection changes
        def update_categories():
            st.session_state.current_categories = st.session_state.sidebar_categories_key
        
        # Multi-select widget for categories
        selected_categories = st.multiselect(
            "Select Categories",
            options=category_options,
            default=st.session_state.current_categories,
            on_change=update_categories,
            key="sidebar_categories_key"
        )
        
        # Filter data based on selection
        filtered_data = sample_data[sample_data['Category'].isin(selected_categories)]
        
        # Display category counts
        category_counts = filtered_data['Category'].value_counts().to_dict()
        category_info = ", ".join([f"{cat} ({count})" for cat, count in category_counts.items()])
        st.write(f"Showing data for: {category_info}")
        
        # Data calculation controls
        st.subheader("Data Calculations")
        
        # Select column for calculation
        col_for_calc = st.selectbox(
            "Select column",
            sample_data.select_dtypes(include=['number']).columns,
            key="calc_column"
        )
        
        # Select operation type
        calc_operation = st.selectbox(
            "Select operation",
            ["Sum", "Count", "Average", "Max", "Min"],
            key="calc_operation"
        )
        
        # Calculate and display result
        if selected_categories and col_for_calc:
            if calc_operation == "Sum":
                result = filtered_data[col_for_calc].sum()
                label = "Sum"
            elif calc_operation == "Count":
                result = filtered_data[col_for_calc].count()
                label = "Count"
            elif calc_operation == "Average":
                result = filtered_data[col_for_calc].mean()
                label = "Average"
            elif calc_operation == "Max":
                result = filtered_data[col_for_calc].max()
                label = "Maximum"
            elif calc_operation == "Min":
                result = filtered_data[col_for_calc].min()
                label = "Minimum"
            
            # Format the result
            if isinstance(result, (int, float)):
                if result >= 1000:
                    formatted_result = f"${result:,.2f}" if "Profit" in col_for_calc or "Sales" in col_for_calc else f"{result:,.2f}"
                else:
                    formatted_result = f"${result:.2f}" if "Profit" in col_for_calc or "Sales" in col_for_calc else f"{result:.2f}"
            else:
                formatted_result = str(result)
            
            # Display result
            st.markdown(f"""
            <div style="margin-top:10px; padding:10px; background-color:#f0f2f6; border-radius:5px; border-left:4px solid {theme_color};">
                <span style="font-size:14px; color:#555;">{label} of {col_for_calc} ({len(filtered_data)} rows):</span>
                <br>
                <span style="font-size:20px; font-weight:bold;">{formatted_result}</span>
            </div>
            """, unsafe_allow_html=True)
        else:
            st.warning("Please select at least one category and a column for calculation")
        
        # Data download option
        st.subheader("Download Data")
        st.markdown(download_csv(sample_data), unsafe_allow_html=True)
        
        st.markdown("</div>", unsafe_allow_html=True)
    
    # -----------------------------
    # MAIN CONTENT AREA WITH TABS
    # -----------------------------
    tabs = st.tabs([
        "Dashboard", 
        "Data Explorer", 
        "Visualization", 
        "Time & Date", 
        "Input Components",
        "St Concepts", 
        "Admin"
    ])
    
    # -----------------------------
    # TAB 1: DASHBOARD
    # -----------------------------
    with tabs[0]:
        st.markdown("<h2 class='section-header'>Dashboard Overview</h2>", unsafe_allow_html=True)
        
        # Expandable section example
        with st.expander("About This Application", expanded=True):
            st.markdown("""
            This is a comprehensive reference application that demonstrates all the key features of Streamlit.
            Use this as a guide when implementing Streamlit features in your projects.
            
            The sidebar on the left provides various controls you can interact with.
            """)
        
        # Metrics display example
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Total Sales", f"${sample_data['Sales'].sum():,}")
        with col2:
            st.metric("Customers", f"{sample_data['Customers'].sum():,}")
        with col3:
            st.metric("Profit", f"${sample_data['Profit'].sum():,}")
        
        # Progress indicators example
        st.markdown("<h3 class='section-header'>Progress Indicators</h3>", unsafe_allow_html=True)
        
        # Spinner example
        if st.button("Show Spinner"):
            with st.spinner("Processing data..."):
                time.sleep(2)
            st.success("Done!")
        
        # Data Screener example
        with st.expander("Data Screener"):
            st.markdown("<h3>Data Filtering</h3>", unsafe_allow_html=True)
            
            # Define columns
            col1, col2 = st.columns(2)
            
            # Get data range values
            min_sales_value = float(sample_data['Sales'].min())
            max_sales_value = float(sample_data['Sales'].max())
            
            # Initialize session states
            if 'screener_current_categories' not in st.session_state:
                st.session_state.screener_current_categories = list(sample_data['Category'].unique())
            
            if 'screener_current_sales_range' not in st.session_state:
                st.session_state.screener_current_sales_range = (min_sales_value, max_sales_value)
            
            if 'screener_filtered_data' not in st.session_state:
                st.session_state.screener_filtered_data = sample_data.copy()
            
            if 'filter_changed' not in st.session_state:
                st.session_state.filter_changed = True
            
            # Category selection in column 1
            with col1:
                def update_categories():
                    st.session_state.screener_current_categories = st.session_state.screener_categories_key
                    st.session_state.filter_changed = True
                    
                selected_category = st.multiselect(
                    "Filter by Category",
                    options=list(sample_data['Category'].unique()),
                    default=st.session_state.screener_current_categories,
                    on_change=update_categories,
                    key="screener_categories_key"
                )
            
            # Sales range slider in column 2
            with col2:
                def update_sales_range():
                    st.session_state.screener_current_sales_range = st.session_state.sales_range_slider
                    st.session_state.filter_changed = True
                    
                min_sales, max_sales = st.slider(
                    "Sales Range",
                    min_value=min_sales_value,
                    max_value=max_sales_value,
                    value=st.session_state.screener_current_sales_range,
                    step=1.0,
                    on_change=update_sales_range,
                    key="sales_range_slider"
                )
            
            # Filter the data when filter changes
            if st.session_state.filter_changed:
                # Reset filter changed flag
                st.session_state.filter_changed = False
                
                # Create progress section
                st.markdown("<h4 style='margin-top: 20px;'>Filtering Progress:</h4>", unsafe_allow_html=True)
                
                # Create percentage display
                percentage_display = st.empty()
                percentage_display.markdown(
                    "<div style='background-color: #f0f7fb; padding: 15px; border-radius: 5px; text-align: center;'>"
                    "<span style='font-size: 14px; color: #555;'>Progress:</span><br>"
                    "<span style='font-size: 40px; font-weight: bold; color: #4CAF50;'>0%</span>"
                    "</div>", 
                    unsafe_allow_html=True
                )
                
                # Progress bar
                progress_bar = st.progress(0)
                
                # Status text
                status_text = st.empty()
                status_text.write("Starting to filter data...")
                
                # Get filter values
                categories = selected_category
                min_val, max_val = min_sales, max_sales
                
                # Process each category with progress updates
                total_categories = len(categories)
                if total_categories == 0:
                    status_text.write("No categories selected")
                    progress_bar.progress(100)
                    percentage_display.markdown(
                        "<div style='background-color: #f0f7fb; padding: 15px; border-radius: 5px; text-align: center;'>"
                        "<span style='font-size: 14px; color: #555;'>Progress:</span><br>"
                        "<span style='font-size: 40px; font-weight: bold; color: #4CAF50;'>100%</span>"
                        "</div>", 
                        unsafe_allow_html=True
                    )
                    st.session_state.screener_filtered_data = pd.DataFrame()
                else:
                    filtered_results = []
                    records_found = 0
                    
                    # Process each category
                    for idx, category in enumerate(categories):
                        # Filter data for this category
                        category_data = sample_data[
                            (sample_data['Category'] == category) & 
                            (sample_data['Sales'] >= min_val) & 
                            (sample_data['Sales'] <= max_val)
                        ]
                        
                        # Count records
                        category_count = len(category_data)
                        records_found += category_count
                        
                        # Add to results
                        filtered_results.append(category_data)
                        
                        # Update progress display
                        progress_percent = int((idx + 1) / total_categories * 100)
                        percentage_display.markdown(
                            f"<div style='background-color: #f0f7fb; padding: 15px; border-radius: 5px; text-align: center;'>"
                            f"<span style='font-size: 14px; color: #555;'>Progress:</span><br>"
                            f"<span style='font-size: 40px; font-weight: bold; color: #4CAF50;'>{progress_percent}%</span>"
                            f"</div>", 
                            unsafe_allow_html=True
                        )
                        progress_bar.progress(progress_percent)
                        status_text.write(f"Processing '{category}' ({idx+1}/{total_categories}): Found {category_count} records")
                        time.sleep(0.1)  # Delay to show progress
                    
                    # Combine results
                    if filtered_results:
                        st.session_state.screener_filtered_data = pd.concat(filtered_results)
                        status_text.write(f"✅ Filtering complete: {records_found} records found")
                    else:
                        st.session_state.screener_filtered_data = pd.DataFrame()
                        status_text.write("⚠️ Filtering complete: No records found")
                    
                    # Final progress update
                    progress_bar.progress(100)
                    percentage_display.markdown(
                        "<div style='background-color: #f0f7fb; padding: 15px; border-radius: 5px; text-align: center;'>"
                        "<span style='font-size: 14px; color: #555;'>Progress:</span><br>"
                        "<span style='font-size: 40px; font-weight: bold; color: #4CAF50;'>100%</span>"
                        "</div>", 
                        unsafe_allow_html=True
                    )
            
            # Get filtered data
            screener_filtered_data = st.session_state.screener_filtered_data
            
            # Display filtered data
            st.dataframe(screener_filtered_data)
            
            # Chart visualization
            if not screener_filtered_data.empty:
                st.write("### Filtered Sales by Category")
                chart = px.bar(
                    screener_filtered_data.groupby('Category')['Sales'].sum().reset_index(),
                    x='Category',
                    y='Sales',
                    title=f'Sales by Category (Filtered: ${min_sales:,.2f} to ${max_sales:,.2f})'
                )
                st.plotly_chart(chart, use_container_width=True, key="screener_sales_chart")
    
    # -----------------------------
    # TAB 2: DATA EXPLORER
    # -----------------------------
    with tabs[1]:
        st.markdown("<h2 class='section-header'>Data Explorer</h2>", unsafe_allow_html=True)
        
        # Data view options
        with st.expander("View Sample Data"):
            st.dataframe(sample_data)
        
        with st.expander("Data Statistics"):
            st.write(sample_data.describe())
        
        # Code and JSON display
        with st.expander("Sample Code & JSON"):
            # Code block display
            st.markdown("<h3>Code Block Example</h3>", unsafe_allow_html=True)
            
            code_example = """
            # Sample Python code to process DataFrame
            import pandas as pd
            
            def analyze_data(df):
                # Calculate summary statistics
                summary = df.describe()
                
                # Get totals by category
                category_totals = df.groupby('Category')['Sales'].sum()
                
                # Find top performing regions
                top_regions = df.groupby('Region')[['Sales', 'Profit']].sum().sort_values('Profit', ascending=False)
                
                return summary, category_totals, top_regions
            """
            
            st.code(code_example, language="python")
            
            # JSON display
            st.markdown("<h3>JSON Display Example</h3>", unsafe_allow_html=True)
            
            sample_json = {
                "app_name": "Streamlit Reference",
                "version": "1.0.0",
                "features": [
                    "Data visualization",
                    "Interactive controls",
                    "Real-time updates"
                ],
                "metrics": {
                    "sales": sample_data['Sales'].sum(),
                    "profit": sample_data['Profit'].sum(),
                    "customers": sample_data['Customers'].sum()
                }
            }
            
            st.json(sample_json)
        
        # File upload example
        st.markdown("<h3 class='section-header'>File Upload</h3>", unsafe_allow_html=True)
        
        uploaded_file = st.file_uploader("Upload a CSV or Excel file", type=["csv", "xlsx"])
        
        if uploaded_file is not None:
            try:
                # Process based on file type
                file_extension = uploaded_file.name.split(".")[-1]
                
                if file_extension.lower() == "csv":
                    uploaded_data = pd.read_csv(uploaded_file)
                elif file_extension.lower() == "xlsx":
                    uploaded_data = pd.read_excel(uploaded_file)
                
                st.success(f"File {uploaded_file.name} successfully uploaded!")
                
                # Show preview
                st.write("Preview of uploaded data:")
                st.dataframe(uploaded_data.head())
                
                # Show statistics
                with st.expander("Data Statistics"):
                    st.write(uploaded_data.describe())
                
                # Download processed data
                st.markdown(download_csv(uploaded_data, "processed_data.csv"), unsafe_allow_html=True)
                
            except Exception as e:
                st.error(f"Error: {e}")
    
    # -----------------------------
    # TAB 3: VISUALIZATION
    # -----------------------------
    with tabs[2]:
        st.markdown("<h2 class='section-header'>Data Visualization</h2>", unsafe_allow_html=True)
        
        # Visualization selection
        def on_viz_change():
            pass
            
        viz_type = st.radio(
            "Select Visualization",
            ["Bar Chart", "Line Chart", "Scatter Plot", "Map", "Pie Chart"],
            on_change=on_viz_change,
            key="viz_type_radio"
        )
        st.session_state.viz_type = viz_type
        
        # Axis selection
        col1, col2 = st.columns(2)
        
        with col1:
            x_axis = st.selectbox("Select X-axis", options=sample_data.columns)
        
        with col2:
            if viz_type != "Pie Chart":
                y_axis = st.selectbox("Select Y-axis", options=sample_data.select_dtypes(include=['number']).columns)
        
        # Create different chart types
        if viz_type == "Bar Chart":
            st.subheader("Bar Chart")
            
            # Matplotlib example
            fig, ax = plt.subplots(figsize=(10, 6))
            sample_data.groupby(x_axis)[y_axis].sum().plot(kind='bar', ax=ax)
            ax.set_title(f'{y_axis} by {x_axis}')
            ax.set_ylabel(y_axis)
            st.pyplot(fig)
            
            # Code example
            with st.expander("Show Matplotlib Code"):
                st.code("""
                # Creating a bar chart with Matplotlib
                fig, ax = plt.subplots(figsize=(10, 6))
                sample_data.groupby(x_axis)[y_axis].sum().plot(kind='bar', ax=ax)
                ax.set_title(f'{y_axis} by {x_axis}')
                ax.set_ylabel(y_axis)
                st.pyplot(fig)
                """, language="python")
            
            # Plotly example
            st.subheader("Interactive Bar Chart (Plotly)")
            fig = px.bar(
                sample_data.groupby(x_axis)[y_axis].sum().reset_index(),
                x=x_axis,
                y=y_axis,
                title=f'{y_axis} by {x_axis}'
            )
            st.plotly_chart(fig, use_container_width=True, key="bar_chart_plotly")
            
            # Code example
            with st.expander("Show Plotly Code"):
                st.code("""
                # Creating an interactive bar chart with Plotly
                fig = px.bar(
                    sample_data.groupby(x_axis)[y_axis].sum().reset_index(),
                    x=x_axis,
                    y=y_axis,
                    title=f'{y_axis} by {x_axis}'
                )
                st.plotly_chart(fig, use_container_width=True)
                """, language="python")
                
        elif viz_type == "Line Chart":
            st.subheader("Line Chart")
            
            # Matplotlib example
            fig, ax = plt.subplots(figsize=(10, 6))
            for category in sample_data['Category'].unique():
                data = sample_data[sample_data['Category'] == category]
                ax.plot(data[x_axis], data[y_axis], label=category)
            ax.set_title(f'{y_axis} vs {x_axis} by Category')
            ax.set_xlabel(x_axis)
            ax.set_ylabel(y_axis)
            ax.legend()
            st.pyplot(fig)
            
            # Plotly example
            st.subheader("Interactive Line Chart (Plotly)")
            fig = px.line(
                sample_data,
                x=x_axis,
                y=y_axis,
                color='Category',
                title=f'{y_axis} vs {x_axis} by Category'
            )
            st.plotly_chart(fig, use_container_width=True, key="line_chart_plotly")
            
        elif viz_type == "Scatter Plot":
            st.subheader("Scatter Plot")
            
            color_by = st.selectbox("Color by", options=sample_data.columns)
            
            # Matplotlib example
            fig, ax = plt.subplots(figsize=(10, 6))
            scatter = ax.scatter(
                sample_data[x_axis],
                sample_data[y_axis],
                c=sample_data[color_by].astype('category').cat.codes,
                alpha=0.6
            )
            ax.set_title(f'{y_axis} vs {x_axis} colored by {color_by}')
            ax.set_xlabel(x_axis)
            ax.set_ylabel(y_axis)
            legend1 = ax.legend(*scatter.legend_elements(), title=color_by)
            ax.add_artist(legend1)
            st.pyplot(fig)
            
            # Plotly example
            st.subheader("Interactive Scatter Plot (Plotly)")
            fig = px.scatter(
                sample_data,
                x=x_axis,
                y=y_axis,
                color=color_by,
                size='Customers',
                hover_data=['Region', 'Sales', 'Profit'],
                title=f'{y_axis} vs {x_axis} colored by {color_by}'
            )
            st.plotly_chart(fig, use_container_width=True, key="scatter_plot_plotly")
            
        elif viz_type == "Map":
            st.subheader("Map Visualization")
            
            # Prepare map data
            map_data = sample_data[['Latitude', 'Longitude', 'Sales']]
            map_data = map_data.rename(columns={
                'Latitude': 'latitude',
                'Longitude': 'longitude'
            })
            
            # Basic map
            st.write("Simple Map with st.map:")
            st.map(map_data)
            
            # Advanced map
            st.write("Advanced Map with Plotly:")
            fig = px.scatter_mapbox(
                sample_data,
                lat="Latitude",
                lon="Longitude",
                color="Region",
                size="Sales",
                hover_name="Category",
                hover_data=["Sales", "Profit"],
                zoom=3,
                height=500,
                mapbox_style="carto-positron"
            )
            st.plotly_chart(fig, use_container_width=True, key="map_plotly")
            
            # Code example
            with st.expander("Show Map Code"):
                st.code("""
                # Simple Map with st.map
                map_data = sample_data[['Latitude', 'Longitude', 'Sales']]
                # Convert column names to lowercase for st.map
                map_data = map_data.rename(columns={
                    'Latitude': 'latitude',
                    'Longitude': 'longitude'
                })
                st.map(map_data)
                
                # Advanced Map with Plotly
                fig = px.scatter_mapbox(
                    sample_data,
                    lat="Latitude",
                    lon="Longitude",
                    color="Region",
                    size="Sales",
                    hover_name="Category",
                    hover_data=["Sales", "Profit"],
                    zoom=3,
                    height=500,
                    mapbox_style="carto-positron"
                )
                st.plotly_chart(fig, use_container_width=True)
                """, language="python")
                
        elif viz_type == "Pie Chart":
            st.subheader("Pie Chart")
            
            # Value selection
            value_col = st.selectbox("Select Value Column", options=sample_data.select_dtypes(include=['number']).columns)
            
            # Matplotlib pie chart
            fig, ax = plt.subplots(figsize=(10, 6))
            sample_data.groupby(x_axis)[value_col].sum().plot(
                kind='pie',
                ax=ax,
                autopct='%1.1f%%'
            )
            ax.set_title(f'{value_col} Distribution by {x_axis}')
            ax.set_ylabel('')
            st.pyplot(fig)
            
            # Plotly pie chart
            st.subheader("Interactive Pie Chart (Plotly)")
            fig = px.pie(
                sample_data.groupby(x_axis)[value_col].sum().reset_index(),
                values=value_col,
                names=x_axis,
                title=f'{value_col} Distribution by {x_axis}'
            )
            st.plotly_chart(fig, use_container_width=True, key="pie_chart_plotly")
    
    # -----------------------------
    # TAB 4: TIME & DATE COMPONENTS
    # -----------------------------
    with tabs[3]:
        st.markdown("<h2 class='section-header'>Time & Date Components</h2>", unsafe_allow_html=True)
        
        # Get current date
        today = datetime.now().date()
        
        # Date range selection
        st.markdown("<h3>Date Range Selection</h3>", unsafe_allow_html=True)
        start_date, end_date = st.date_input(
            "Select a date range for summary",
            [today - timedelta(days=30), today],
            min_value=today - timedelta(days=365),
            max_value=today + timedelta(days=365)
        )
        
        st.write(f"Selected range: {start_date} to {end_date}")
        
        # Category summary by date
        st.markdown("<h3>Category Summary by Date Range</h3>", unsafe_allow_html=True)
    
        # Convert date column to datetime
        if not pd.api.types.is_datetime64_any_dtype(sample_data['Date']):
            sample_data['Date'] = pd.to_datetime(sample_data['Date'])
        
        # Filter data by date range
        date_filtered_data = sample_data[
            (sample_data['Date'] >= pd.Timestamp(start_date)) & 
            (sample_data['Date'] <= pd.Timestamp(end_date))
        ]
    
        # Create summary if data exists
        if not date_filtered_data.empty:
            # Aggregate by category
            category_summary = date_filtered_data.groupby('Category').agg({
                'Sales': 'sum',
                'Profit': 'sum',
                'Customers': 'sum'
            }).reset_index()
            
            # Display summary table
            st.write(f"### Summary for period: {start_date} to {end_date}")
            st.dataframe(category_summary)
            
            # Visualization
            fig = px.bar(
                category_summary,
                x='Category',
                y=['Sales', 'Profit'],
                barmode='group',
                title=f'Category Performance ({start_date} to {end_date})'
            )
            st.plotly_chart(fig, use_container_width=True, key="date_category_chart")
        else:
            st.info("No data available for the selected date range.")
        
        # Date input examples
        col1, col2 = st.columns(2)
        
        with col1:
            st.markdown("<h3>Date Input</h3>", unsafe_allow_html=True)
            selected_date = st.date_input(
                "Select a date",
                today,
                min_value=today - timedelta(days=365),
                max_value=today + timedelta(days=365),
                key="single_date_input"
            )
            st.write(f"You selected: {selected_date}")
        
        with col2:
            # Current timestamp display
            st.markdown("<h3>Current Date & Time</h3>", unsafe_allow_html=True)
            st.write(f"Current timestamp: {datetime.now()}")
    
    # -----------------------------
    # TAB 5: INPUT COMPONENTS
    # -----------------------------
    with tabs[4]:
        st.markdown("<h2 class='section-header'>Input Components</h2>", unsafe_allow_html=True)
        
        # Form submission example
        st.markdown("<h3>Form Submission</h3>", unsafe_allow_html=True)
        
        with st.form(key="input_form"):
            st.write("User Information Form")
            
            col1, col2 = st.columns(2)
            
            with col1:
                user_name = st.text_input("Name")
                user_email = st.text_input("Email")
                user_dob = st.date_input("Date of Birth")
            
            with col2:
                user_category = st.selectbox("Category", ["Individual", "Business", "Government", "Educational"])
                user_rating = st.slider("Rate our service", 1, 10, 5)
                user_newsletter = st.checkbox("Subscribe to newsletter")
            
            user_comments = st.text_area("Additional Comments")
            
            submit_button = st.form_submit_button(label="Submit")
        
        # Display form results
        if submit_button:
            st.success("Form submitted successfully!")
            
            results = {
                "Name": user_name,
                "Email": user_email,
                "Date of Birth": user_dob,
                "Category": user_category,
                "Rating": user_rating,
                "Newsletter": user_newsletter,
                "Comments": user_comments
            }
            
            st.write("Form Submission Results:")
            st.json(results)
    
    # -----------------------------
    # TAB 6: STREAMLIT CONCEPTS
    # -----------------------------
    with tabs[5]:
        st.markdown("<h2 class='section-header'>Advanced Features</h2>", unsafe_allow_html=True)
        
        # Session state management
        st.markdown("<h3>Session State Management</h3>", unsafe_allow_html=True)
        
        with st.expander("About Session State", expanded=True):
            st.markdown("""
            Session state is used to preserve values across reruns, enabling complex app behavior like
            multi-page workflows, form validation, and preserving user inputs.
            """)
        
        # Initialize session state
        if 'counter' not in st.session_state:
            st.session_state.counter = 0
        
        if 'todo_list' not in st.session_state:
            st.session_state.todo_list = []
        
        # Counter example
        col1, col2 = st.columns(2)
        
        with col1:
            st.subheader("Counter Example")
            st.write(f"Current count: {st.session_state.counter}")
            
            if st.button("Increment Counter"):
                st.session_state.counter += 1
                st.rerun()
            
            if st.button("Reset Counter"):
                st.session_state.counter = 0
                st.rerun()
        
        # To-do list example
        with col2:
            st.subheader("To-Do List Example")
            
            new_todo = st.text_input("Add a new to-do item")
            
            if st.button("Add Item"):
                if new_todo:
                    st.session_state.todo_list.append({"text": new_todo, "done": False})
                    st.rerun()
            
            # Display to-do list
            for i, todo in enumerate(st.session_state.todo_list):
                col1, col2 = st.columns([4, 1])
                
                with col1:
                    if todo["done"]:
                        st.markdown(f"~~{todo['text']}~~")
                    else:
                        st.write(todo["text"])
                
                with col2:
                    if st.button("Done" if not todo["done"] else "Undo", key=f"todo_{i}"):
                        st.session_state.todo_list[i]["done"] = not st.session_state.todo_list[i]["done"]
                        st.rerun()
        
        # Caching example
        st.markdown("<h3 class='section-header'>Caching</h3>", unsafe_allow_html=True)
        
        with st.expander("About Caching", expanded=True):
            st.markdown("""
            Streamlit provides caching to improve app performance by storing the results of expensive computations.
            
            There are three main types of caches:
            
            1. `st.cache_data`: Cache the returned value of a function, which can include data like DataFrames or lists.
            2. `st.cache_resource`: Cache resources like database connections or ML models that shouldn't be duplicated.
            3. `@st.experimental_memo`: Deprecated, use `st.cache_data` instead.
            """)
        
        st.subheader("Cached Function Example")
        
        operation = st.selectbox(
            "Select operation to perform on data",
            ["sum", "mean", "count", "describe"]
        )
        
        if st.button("Process Data (Cached)"):
            with st.spinner("Processing data..."):
                # Use cached function
                result = process_data(sample_data, operation)
                st.write("Result:", result)
                st.info("This result is cached. Running it again will be much faster!")
        
        # HTML embedding example
        st.markdown("<h3 class='section-header'>HTML & iframe Embedding</h3>", unsafe_allow_html=True)
        
        with st.expander("HTML Embedding Examples"):
            # Custom HTML
            st.markdown("<h4>Custom HTML Elements</h4>", unsafe_allow_html=True)
            
            custom_html = """
            <div style="padding: 15px; background-color: #f0f0f0; border-radius: 10px;">
                <h4 style="color: #2196F3;">Custom HTML Content</h4>
                <p>This is a custom HTML component embedded in Streamlit.</p>
                <ul>
                    <li>You can use any HTML tags</li>
                    <li>Apply custom CSS styling</li>
                    <li>Create complex layouts</li>
                </ul>
                <button style="background-color: #4CAF50; color: white; padding: 8px 16px; border: none; border-radius: 4px; cursor: pointer;">
                    Example Button (non-functional)
                </button>
            </div>
            """
            
            st.markdown(custom_html, unsafe_allow_html=True)
            
            # iframe example
            st.markdown("<h4>iframe Example</h4>", unsafe_allow_html=True)
            
            iframe_html = """
            <iframe 
                src="https://docs.streamlit.io"
                width="100%" 
                height="450" 
                style="border: 1px solid #ddd; border-radius: 5px;"
                allowfullscreen>
            </iframe>
            """
            
            st.markdown(iframe_html, unsafe_allow_html=True)
            
            # Code example
            show_code = st.checkbox("Show Embedding Code", key="show_html_code")
            if show_code:
                st.code("""
                # Basic HTML embedding
                custom_html = '''
                <div style="padding: 15px; background-color: #f0f0f0; border-radius: 10px;">
                    <h4 style="color: #2196F3;">Custom HTML Content</h4>
                    <p>This is a custom HTML component embedded in Streamlit.</p>
                    <!-- More HTML content -->
                </div>
                '''
                
                st.markdown(custom_html, unsafe_allow_html=True)
                
                # iframe embedding
                iframe_html = '''
                <iframe 
                    src="https://docs.streamlit.io"
                    width="100%" 
                    height="450" 
                    style="border: 1px solid #ddd; border-radius: 5px;"
                    allowfullscreen>
                </iframe>
                '''
                
                st.markdown(iframe_html, unsafe_allow_html=True)
                """, language="python")
    
    # -----------------------------
    # TAB 7: ADMIN PANEL
    # -----------------------------
    with tabs[6]:
        st.markdown("<h2 class='section-header'>Admin Panel</h2>", unsafe_allow_html=True)
        
        # Role-based access control
        if st.session_state.role != "admin":
            st.warning("You need admin privileges to access this section")
        else:
            st.success("Welcome to the Admin Panel")
            
            # Admin tabs
            admin_tabs = st.tabs(["User Management", "Email Center", "System Monitoring"])
            
            # User Management Tab
            with admin_tabs[0]:
                st.markdown("<h3>User List</h3>", unsafe_allow_html=True)
                
                # Get current users
                current_users = load_users()
                
                # Display user table
                user_data = [
                    {"username": username, "role": data["role"], "email": data.get("email", "")} 
                    for username, data in current_users.items()
                ]
                
                st.table(pd.DataFrame(user_data))
                
                # Add new user form
                st.markdown("<h3>Add New User</h3>", unsafe_allow_html=True)
                
                with st.form("add_user_form"):
                    new_username = st.text_input("Username")
                    new_password = st.text_input("Password", type="password")
                    new_role = st.selectbox("Role", ["user", "admin"])
                    new_email = st.text_input("Email")
                    
                    submit_user = st.form_submit_button("Add User")
                
                if submit_user:
                    if new_username and new_password:
                        # Add user to file
                        success = add_user_to_file(new_username, new_password, new_role, new_email)
                        if success:
                            # Reload users
                            USERS = load_users()
                            st.success(f"User {new_username} added successfully!")
                            
                            # Show updated list
                            updated_user_data = [
                                {"username": username, "role": data["role"], "email": data.get("email", "")} 
                                for username, data in USERS.items()
                            ]
                            st.table(pd.DataFrame(updated_user_data))
                        else:
                            st.error(f"Username '{new_username}' already exists!")
                    else:
                        st.error("Username and password are required")
            
            # Email Center Tab
            with admin_tabs[1]:
                st.markdown("<h3>Send Email to Users</h3>", unsafe_allow_html=True)
                
                # Get all user emails
                current_users = load_users()
                all_emails = [data.get("email", "") for data in current_users.values() if data.get("email", "")]
                
                # Email form
                with st.form("email_form"):
                    # Choose recipients
                    email_to_all = st.checkbox("Send to all users")
                    
                    if email_to_all:
                        recipients = all_emails
                        st.write(f"Email will be sent to {len(recipients)} users")
                    else:
                        # Individual selection
                        email_options = [(username, data.get("email", "")) 
                                        for username, data in current_users.items() 
                                        if data.get("email", "")]
                        
                        selected_users = st.multiselect(
                            "Select recipients",
                            options=[f"{username} ({email})" for username, email in email_options],
                            default=[]
                        )
                        
                        # Extract emails
                        recipients = [email for username, email in email_options 
                                    if f"{username} ({email})" in selected_users]
                    
                    # Email content
                    email_subject = st.text_input("Subject")
                    email_body = st.text_area("Message", height=150)
                    
                    # Submit button
                    send_button = st.form_submit_button("Send Email")
                
                if send_button:
                    if not recipients:
                        st.error("Please select at least one recipient")
                    elif not email_subject or not email_body:
                        st.error("Subject and message are required")
                    else:
                        # Send email
                        result = send_dummy_email(recipients, email_subject, email_body)
                        st.success(result)
                
                # Email history
                if 'sent_emails' in st.session_state and st.session_state.sent_emails:
                    st.markdown("<h3>Email History</h3>", unsafe_allow_html=True)
                    
                    for i, email in enumerate(reversed(st.session_state.sent_emails)):
                        with st.expander(f"{email['subject']} - {email['timestamp']}"):
                            st.write(f"**Time:** {email['timestamp']}")
                            st.write(f"**To:** {', '.join(email['recipients'])}")
                            st.write(f"**Subject:** {email['subject']}")
                            st.write("**Message:**")
                            st.text(email['message'])
            
            # System Monitoring Tab
            with admin_tabs[2]:
                st.markdown("<h3>System Status</h3>", unsafe_allow_html=True)
                
                # System metrics
                col1, col2, col3 = st.columns(3)
                
                with col1:
                    cpu_usage = random.randint(10, 95)
                    st.metric("CPU Usage", f"{cpu_usage}%", f"{random.choice(['+', '-'])}{random.randint(1, 10)}%")
                    st.progress(cpu_usage / 100)
                
                with col2:
                    memory_usage = random.randint(20, 85)
                    st.metric("Memory Usage", f"{memory_usage}%", f"{random.choice(['+', '-'])}{random.randint(1, 8)}%")
                    st.progress(memory_usage / 100)
                
                with col3:
                    disk_usage = random.randint(30, 75)
                    st.metric("Disk Usage", f"{disk_usage}%", f"{random.choice(['+', '-'])}{random.randint(1, 5)}%")
                    st.progress(disk_usage / 100)
                
                # Active users chart
                st.markdown("<h3>Active Users</h3>", unsafe_allow_html=True)
                
                # Generate sample data
                dates = pd.date_range(end=datetime.now(), periods=30, freq='D')
                active_users = np.random.randint(50, 200, size=30)
                
                # Plotly line chart
                fig = px.line(
                    x=dates,
                    y=active_users,
                    labels={'x': 'Date', 'y': 'Active Users'},
                    title='Active Users (Last 30 Days)'
                )
                
                st.plotly_chart(fig, use_container_width=True, key="admin_users_chart")

# Run the main application
if __name__ == "__main__":
    main()

Overwriting streamlit_application.py


In [3]:
# Run the application
!streamlit run streamlit_application.py

^C


## Component Guide

### Basic Components

- **Text Output**: `st.title()`, `st.header()`, `st.markdown()`, `st.write()`
- **Data Display**: `st.dataframe()`, `st.table()`, `st.json()`
- **Media**: `st.image()`, `st.audio()`, `st.video()`

### Input Widgets

- **Text Input**: `st.text_input()`, `st.text_area()`, `st.number_input()`
- **Selection**: `st.checkbox()`, `st.radio()`, `st.selectbox()`, `st.multiselect()`
- **Buttons**: `st.button()`, `st.download_button()`
- **Date/Time**: `st.date_input()`, `st.time_input()`
- **Sliders**: `st.slider()`, `st.select_slider()`
- **File Upload**: `st.file_uploader()`
- **Color Picker**: `st.color_picker()`

### Layout Elements

- **Columns**: `st.columns()`
- **Tabs**: `st.tabs()`
- **Expanders**: `st.expander()`
- **Containers**: `st.container()`
- **Sidebar**: `st.sidebar`

### Progress & Status

- **Progress Bar**: `st.progress()`
- **Spinner**: `st.spinner()`
- **Status**: `st.success()`, `st.error()`, `st.warning()`, `st.info()`
- **Metrics**: `st.metric()`

### Advanced Features

- **Forms**: `st.form()`, `st.form_submit_button()`
- **Session State**: `st.session_state`
- **Caching**: `@st.cache_data`, `@st.cache_resource`
- **HTML/Components**: `st.markdown(..., unsafe_allow_html=True)`
- **Configuration**: `st.set_page_config()`

## Requirements and Additional Files

To make this reference complete, you'll also need:

1. **requirements.txt** file with: streamlit>=1.28.0
pandas>=2.0.0
numpy>=1.24.0
matplotlib>=3.7.0
plotly>=5.15.0
pillow>=10.0.0

2. **README.md** file with documentation that explains:
   - Overview of the application
   - How to install and run
   - Features reference
   - Explanation of example cckage.